### Lora Trainer Remaked by layla-focalors  
This is based on thr work of Hollowstrawberry. Thank you

### Running Training

In [4]:
import os
import re
import toml
import shutil
import zipfile
from time import time

# 프로젝트 이름은 이미지가 포함된 폴더와 동일합니다. 공백은 허용되지 않습니다.

# 수정 가능
PROJECT_NAME = 'Project'

# 수정을 권장하지 않음
FOLDER_STRUCTURE = f'/Loras/{PROJECT_NAME}/dataset'
TRAINING_MODEL = 'https://huggingface.co/Lykon/AnyLoRA/resolve/main/AnyLoRA_noVae_fp16-pruned.ckpt'

# 512의 해상도는 Stable Diffusion 1.5의 표준입니다. 고해상도 학습은 훨씬 느리지만 더 나은 세부 정보로 이어질 수 있습니다.

# 최상의 결과를 얻기 위해 학습하는 동안 이미지의 크기가 자동으로 조정되므로 직접 자르거나 크기를 조정할 필요가 없습니다.

RESOLUTION = 512

# 이 옵션은 추가 비용 없이 이미지를 정상적으로 훈련하고 뒤집어 더 많은 것을 학습합니다. 이미지가 20개 미만인 경우 특별히 켜십시오.

# Lora의 비대칭 요소에 관심이 있다면 끄십시오.

FLIP_AUG = False

# 이미지는 훈련 중에 이 횟수만큼 반복됩니다. 이미지에 반복을 곱한 값은 200에서 400 사이가 좋습니다.

NUM_REPEATS = 10

# 훈련할 기간을 선택합니다. 좋은 시작점은 약 10 epoch 또는 약 2000 단계입니다.

# 하나의 Epoch는 이미지 수에 반복을 곱한 후 배치 크기로 나눈 값과 같은 단계입니다.

PREFERRED_UNIT = 'Epoches'
PREFFERED_DURATION = 10

# 배치 크기를 늘리면 학습 속도가 빨라지지만 학습 속도가 느려질 수 있습니다. 권장 2 또는 3.

TRAIN_BATCH_SIZE = 2

# 학습률은 결과에 가장 중요합니다. 영상이 많을 때 더 느리게 훈련시키고 싶거나 dim과 alpha가 높은 경우에는 unet을 2e-4 이하로 옮기십시오.

# 텍스트 인코더는 Lora가 개념을 약간 더 잘 배울 수 있도록 도와줍니다. unet의 절반 또는 5 분의 1로 만드는 것이 좋습니다. 스타일을 훈련하는 경우 0으로 설정할 수도 있습니다.

UNET_LR = '5e-4'
TEXT_ENCODER_LR = '1e-4'

# Code
# Create Config

XFORMERS = True
COMMIT = "9a67e0df390033a89f17e70df5131393692c2a55"
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True
project_name = PROJECT_NAME
custom_model_is_based_on_sd2 = False
resolution = RESOLUTION
flip_aug = FLIP_AUG
num_repeats = NUM_REPEATS
preferred_unit = PREFERRED_UNIT
preferred_duration = PREFFERED_DURATION
train_batch_size = TRAIN_BATCH_SIZE
unet_lr = UNET_LR
text_encoder_lr = TEXT_ENCODER_LR

caption_extension = ".txt"
shuffle_tags = True 
shuffle_caption = shuffle_tags
activation_tags = "1" 
keep_tokens = int(activation_tags)

how_many = preferred_unit

max_train_epochs = how_many if preferred_unit == "Epochs" else None
max_train_steps = how_many if preferred_unit == "Steps" else None
save_every_n_epochs = 1

keep_only_last_n_epochs = 10
if not save_every_n_epochs:
  save_every_n_epochs = max_train_epochs
if not keep_only_last_n_epochs:
  keep_only_last_n_epochs = max_train_epochs

lr_scheduler = "cosine_with_restarts"
lr_scheduler_number = 3
lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0

lr_warmup_ratio = 0
lr_warmup_steps = 0

lora_type = "LoRA"
network_dim = 16 
network_alpha = 8

conv_dim = 8
conv_alpha = 4

optimizer = "AdamW8bit"
override_values_for_dadapt_and_prodigy = True

network_module = "networks.lora"
network_args = None
if lora_type.lower() == "locon":
  network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]

if optimizer.lower() == "prodigy" or "dadapt" in optimizer.lower():
  if override_values_for_dadapt_and_prodigy:
    unet_lr = 0.5
    text_encoder_lr = 0.5
    lr_scheduler = "constant_with_warmup"
    lr_warmup_ratio = 0.05
    network_alpha = network_dim
  if not optimizer_args:
    optimizer_args = ["decouple=True","weight_decay=0.01","betas=[0.9,0.999]"]
    if optimizer == "Prodigy":
      optimizer_args.extend(["d_coef=2","use_bias_correction=True"])
      if lr_warmup_ratio > 0:
        optimizer_args.append("safeguard_warmup=True")
      else:
        optimizer_args.append("safeguard_warmup=False")



    




